## Callysto Notebook Tagger (Java, HTML, iPython, Geogebra)

**Description**: This is a notebook that iterates through directory files and returns which cells contain javascript, html, ipython widgets, or geogebra references. It is being used for Jupyterlab migration and general bugfixing.

**Usage**: Run this notebook in the parent directory containing the notebooks or in the directory itself. run langtag() before clean_langtag()

**Notes**: Generates raw log for read/writing - not human readable. Also not optimized but good enough for my purposes.

Last Edited: June 30, 2020

Author: LNC

Contact: lisa.cao@cybera.ca

In [ ]:
#import libraries
import os 
import re
import json

In [ ]:
#create datafame - for later feature
#import pandas as pd
#df = pd.DataFrame(columns=['Javascript', 'Extension Needed', 'Geogebra', 'HTML'], index =["Notebook"])

In [ ]:
# use regex to detect language references
def cell_tagger(contents): 
    java = r"java|javascript|\.js"
    html = r"html|.(\/\>)"
    ggb = r"ggb|geogebra"
    lib = r"ipython|iplot|qgrid" 
    tagsdict = {"Java": java, #dictionary for more readable tags 
                "HTML": html,
                "Geogebra": ggb, 
                "iPython, iPlot, or qgrid": lib}
    printout = []
    for tag, code in tagsdict.items(): # iterate and search 
        regex = re.compile(code)
        if bool(re.search(code, contents)) == True: # find all instances   
            printout.append(tag)
        else: 
            pass
    return printout

In [ ]:
# search through all source code in cells and detect if contains language 
def langtag():
    for root, dirs, files in os.walk("."):
        for filename in files:
            if filename.endswith('.ipynb'): # select notebooks
                file = os.path.join(root, filename)
                notebook = json.load(open(file)) # load notebook json
                cell_number = 0
                file_contains = []
                with open("celltag_output.txt", "a") as out: 
                    for cell in notebook['cells']:
                        cell_number += 1 # cell counter for output
                        try:
                            cell_contents = cell['source'][0] # parse json
                            cell_tags = cell_tagger(cell_contents)
                            if cell_tags:
                                file_contains.append([cell_number, cell_tags])
                            else:
                                pass
                        except:
                            pass
                    if file_contains != []:
                        print(filename, "cell flags", file_contains, "\n", file = out)
                out.close()

In [ ]:
# clean up output
def clean_langtag(): 
    unique = set() # place for unique lines
    with open("celltag_output.txt", 'r') as file: 
        for line in file: 
            if line not in unique:
                unique.add(line)
        file.close()
    return unique

In [ ]:
# as a clean function
def tag_cells():
    langtag()
    clean_langtag()
    print("Complete, see celltag_output.txt")

In [ ]:
# execute
tag_cells()